# Tutorial HANNAH Framework 
## AutoML School
### Hannover 2024 


### Setup tooling

In [1]:
#!pip install  hannah[vision,models]@git+https://github.com/ekut-es/hannah.git

### Setup library

In [2]:
import hannah
import pathlib
import torch
import os
from contextlib import contextmanager
import sys



# Modify the python module search path to include the current directory
sys.path.append(str(pathlib.Path.cwd()))


@contextmanager
def change_wd(path: pathlib.Path, create: bool = True, clean: bool = False):
    current = pathlib.Path.cwd()
    
    if clean and path.exists():
        import shutil
        shutil.rmtree(path)
        path.mkdir(parents=True)
    
    if create and not path.exists():
        path.mkdir(parents=True)    
    
    try:
        os.chdir(path)
        yield
    finally:
        os.chdir(current)
    
    


### Create Search Space
First we create a simple search space. This search space description is written to a file such that the `hydra` config framework
can refer to it and instantiate the search space. 

The search space consists of a variable number of `conv_relu` blocks with a `linear` layer at the end. 



In [3]:
%%writefile space.py
from hannah.nas.functional_operators.op import ChoiceOp, Tensor, scope
from hannah.nas.functional_operators.operators import AdaptiveAvgPooling, Conv2d, Linear, Relu
from hannah.nas.functional_operators.op import search_space
from hannah.nas.parameters.parameters import CategoricalParameter, IntScalarParameter


@scope
def conv_relu(input, kernel_size, out_channels, stride):
    in_channels = input.shape()[1]
    weight = Tensor(name='weight',
                    shape=(out_channels, in_channels, kernel_size, kernel_size),
                    axis=('O', 'I', 'kH', 'kW'),
                    grad=True)

    conv = Conv2d(stride=stride)(input, weight)
    relu = Relu()(conv)
    return relu


def adaptive_avg_pooling(input):
    return AdaptiveAvgPooling()(input)


def dynamic_depth(*exits, switch):
    return ChoiceOp(*exits, switch=switch)()


def linear(input, out_features):
    input_shape = input.shape()
    in_features = input_shape[1] * input_shape[2] * input_shape[3]
    weight = Tensor(name='weight',
                    shape=(in_features, out_features),
                    axis=('in_features', 'out_features'),
                    grad=True)

    out = Linear()(input, weight)
    return out


@scope
def classifier_head(input, num_classes):
    out = adaptive_avg_pooling(input)
    out = linear(out, num_classes)
    return out


@search_space
def cnn_search_space(name, input, max_blocks=3, max_channels=64, num_classes=10):
    out_channels = IntScalarParameter(16, max_channels, step_size=8, name="out_channels")
    kernel_size = CategoricalParameter([3, 5, 7, 9], name="kernel_size")
    stride = CategoricalParameter([1, 2], name="stride")
    num_blocks = IntScalarParameter(1, max_blocks, name="num_blocks")
    exits = []
    out = input
    for i in range(num_blocks.max + 1):
        out = conv_relu(out,
                        stride=stride.new(),
                        out_channels=out_channels.new(),
                        kernel_size=kernel_size.new())
        exits.append(out)

    out = dynamic_depth(*exits, switch=num_blocks)
    out = classifier_head(out, num_classes=num_classes)
    return out

Overwriting space.py


Now we create a config file for `hydra`:

In [4]:
from omegaconf import OmegaConf
import os

cfg_dict ={"_target_": "space.cnn_search_space",
           "name": "cnn_search_space",
           "num_classes": 10,
           "max_channels": 64,
           "max_blocks": 2}

cfg = OmegaConf.create(cfg_dict)

if not os.path.exists("model"):
    os.mkdir("model")
OmegaConf.save(config=cfg, f="model/cnn_search_space.yaml")


### Define Hardware Description

### Configure NAS

### Run NAS

Now we can run the NAS in the "terminal". Note, that we have to append the current working directory to the PYTHONPATH such 
that the search space located in our current directory can be found. If the model file where to be located in the `hannah/models/`
directory that would not be necessary.

In [10]:
from hannah.tools.train import main
from hydra import compose, initialize_config_module

wd = pathlib.Path.cwd()

with initialize_config_module("hannah.conf", version_base="1.2"):
    cfg = compose(config_name="config", overrides=["model=cnn_search_space", "nas=random_nas", "trainer.max_epochs=1", "dataset=mnist", f"dataset.data_folder={wd / 'data'}", "+trainer.enable_progress_bar=false"])

    from rich import inspect
    inspect(cfg)    
    with change_wd(wd / "run1", create=True, clean=True):
        main(cfg)

╭─────────────────────────────────── <class 'omegaconf.dictconfig.DictConfig'> ───────────────────────────────────╮
│ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {'dataset': {'data_folder': '/local/gerum/hannah/tutorials/automlschool24/data', 'cls':                     │ │
│ │ 'hannah.datasets.vision.MNISTDataset', 'dataset': 'mnist', 'val_percent': 0.1}, 'features': {'_target_':    │ │
│ │ 'torch.nn.Identity'}, 'model': {'_target_': 'space.cnn_search_space', 'name': 'cnn_search_space',           │ │
│ │ 'num_classes': 10, 'max_channels': 64, 'max_blocks': 2}, 'scheduler': {'_target_':                          │ │
│ │ 'torch.optim.lr_scheduler.OneCycleLR', 'max_lr': 0.001, 'pct_start': 0.3, 'anneal_strategy': 'cos',         │ │
│ │ 'cycle_momentum': True, 'base_momentum': 0.85, 'max_momentum': 0.95, 'div_factor': 25.0,                    │ │
│ │ 'final_div_factor': 10000.0, 'last_epoch': -1}, 'optimizer': {'_target_': 'torch.optim.adamw.AdamW', 'lr':  │ │
│ │ 0.001, 'betas': [0.9, 0.999], 'eps': 1e-08, 'weight_decay': 1e-05, 'amsgrad': False}, 'module':             │ │
│ │ {'_target_': 'hannah.modules.vision.ImageClassifierModule', 'num_workers': 0, 'batch_size': 128,            │ │
│ │ 'shuffle_all_dataloaders': False}, 'trainer': {'_target_': 'pytorch_lightning.trainer.Trainer',             │ │
│ │ 'accelerator': 'auto', 'devices': 1, 'limit_train_batches': 1.0, 'limit_val_batches': 1.0,                  │ │
│ │ 'limit_test_batches': 1.0, 'max_epochs': 1, 'default_root_dir': '.', 'fast_dev_run': False,                 │ │
│ │ 'overfit_batches': 0.0, 'benchmark': False, 'deterministic': 'warn', 'gradient_clip_val': 0,                │ │
│ │ 'accumulate_grad_batches': 1, 'plugins': None, 'strategy': 'auto', 'reload_dataloaders_every_n_epochs': 0,  │ │
│ │ 'precision': 32, 'enable_progress_bar': False}, 'checkpoint': {'_target_':                                  │ │
│ │ 'pytorch_lightning.callbacks.ModelCheckpoint', 'dirpath': 'checkpoints', 'save_top_k': 1, 'verbose': True,  │ │
│ │ 'monitor': 'val_error', 'mode': 'min', 'save_last': True}, 'nas': {'sampler': {'_target_':                  │ │
│ │ 'hannah.nas.search.sampler.random_sampler.RandomSampler'}, 'model_trainer': {'_target_':                    │ │
│ │ 'hannah.nas.search.model_trainer.simple_model_trainer.SimpleModelTrainer', 'per_process_memory_fraction':   │ │
│ │ None}, 'constraint_model': {'_target_': 'hannah.nas.constraints.constraint_model.ConstraintModel',          │ │
│ │ 'method': 'naive', 'fix_strategy': 'last', 'timeout': 300}, 'predictor': {'gcn': {'_target_':               │ │
│ │ 'hannah.nas.performance_prediction.simple.GCNPredictor', 'model': {'_target_':                              │ │
│ │ 'hannah.nas.performance_prediction.gcn.predictor.GaussianProcessPredictor', 'input_feature_size': 35}},     │ │
│ │ 'macs': {'_target_': 'hannah.nas.performance_prediction.simple.MACPredictor', 'predictor': 'fx'}},          │ │
│ │ 'presampler': {'_target_': 'hannah.nas.search.presampler.simple.SingleRangeChecker', 'min': 250000, 'max':  │ │
│ │ 550000}, '_target_': 'hannah.nas.search.search.DirectNAS', 'budget': 5, 'n_jobs': 10, 'total_candidates':   │ │
│ │ 5, 'num_selected_candidates': 10, 'presample': True, 'bounds': {'val_error': 0.1, 'total_macs': 128000000,  │ │
│ │ 'total_weights': 500000}}, 'experiment_id': 'test', 'output_dir': 'trained_models', 'auto_lr': False,       │ │
│ │ 'resume': False, 'fx_mac_summary': True, 'skip_test': False, 'skip_val': False, 'seed': [1234],             │ │
│ │ 'validate_output': False}                                                                                   │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                                 │
│         auto_lr = False                               


 **                                            **
/**                                           /**
/**       ******   *******  *******   ******  /**
/******  //////** //**///**//**///** //////** /******
/**///**  *******  /**  /** /**  /**  ******* /**///**
/**  /** **////**  /**  /** /**  /** **////** /**  /**
/**  /**//******** ***  /** ***  /**//********/**  /**
//   //  //////// ///   // ///   //  //////// //   //

dataset:
  data_folder: /local/gerum/hannah/tutorials/automlschool24/data
  cls: hannah.datasets.vision.MNISTDataset
  dataset: mnist
  val_percent: 0.1
features:
  _target_: torch.nn.Identity
model:
  _target_: space.cnn_search_space
  name: cnn_search_space
  num_classes: 10
  max_channels: 64
  max_blocks: 2
scheduler:
  _target_: torch.optim.lr_scheduler.OneCycleLR
  max_lr: 0.001
  pct_start: 0.3
  anneal_strategy: cos
  cycle_momentum: true
  base_momentum: 0.85
  max_momentum: 0.95
  div_factor: 25.0
  final_div_factor: 10000.0
  last_epoch: -1
optimizer:
  _tar

### Evaluate

In [6]:
!hannah-nas-eval force=True

{'data': {'nas': '.'}, 'metrics': {'total_act': {'name': 'Activations'}, 'total_weights': {'name': 'Weights'}, 'weights_m': {'name': 'Weights [M]', 'derived': 'data["total_weights"] / 1000 / 1000'}, 'val_accuracy': {'name': 'Accuracy [%]', 'derived': '(1.0 - data["val_error"]) * 100.0'}, 'act_k': {'name': 'Activations [k]', 'derived': 'data["total_act"] / 1000'}, 'macs_m': {'name': 'MACS [M]', 'derived': 'data["total_macs"] / 1000 / 1000'}}, 'plots': [{'type': 'comparison', 'name': 'accuracy_memory', 'metrics': ['val_accuracy', 'weights_m', 'act_k']}, {'type': 'comparison', 'name': 'accuracy_macs', 'metrics': ['val_accuracy', 'macs_m']}], 'extract': {'nas': {'bounds': {'val_error': 0.02}}}, 'experiment': 'test', 'force': True}
[2024-09-04 13:28:16,677][nas_eval][INFO] - Current working directory /local/gerum/hannah/tutorials/automlschool24
[2024-09-04 13:28:16,677][nas_eval.prepare][INFO] - Extracting design points
[2024-09-04 13:28:16,677][nas_eval.prepare][INFO] -   Extracting design

![](comparison_accuracy_macs.png)

![](comparison_accuracy_memory.png)

: 